In [2]:
import warnings

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from Functions import pmfg
import seaborn as sns

warnings.filterwarnings('ignore')
%matplotlib inline


# Functions from the model        
def P(x):
    return np.minimum(0.1 + 0.9*x, 1)

def Q(x):
    return 1. - 2**(-x/100.)
   
    
# Function to select from DataFrame
def dfSelecter(df, rows=None, columns=None):
    if rows == None:
        if columns == None:
            return df.copy()
        else:
            try:
                return df.loc[:, columns]
            except KeyError:
                print "Name Error in column."
    else:
        if columns == None:
            try:
                return df.loc[rows]
            except KeyError:
                print "Name Error in row."
        else:
            try:
                return df.loc[rows, columns]
            except KeyError:
                try:
                    df.loc[:, columns]
                except KeyError:
                    print "Name Error in column."
                try:
                    df.loc[rows]
                except KeyError:
                    print "Name Error in row."

                    
# Extract identifier of asset class from column names                    
def ExtractAssetClass(col):
    return col[0:col.find("_")]

def ExtractAssetName(col):
    return col[col.find("_")+1:]


def getRiskWeights(minmax):
    # Initialize dictionaries
    if minmax == "min":
        return {"1": 0.002, "2": 0.5, "3": 0.7, 
                "4-1": 0.4, "4-2": 0.8, "4-3": 1, "5": 1}
    elif minmax == "max":
        return {"1": 0.1, "2": 1, "3": 1.3, 
                "4-1": 0.8, "4-2": 1.2, "4-3": 1.3, "5": 2}
    else:
        # In case of input error, return minimum weights.
        return {"1": 0.002, "2": 0.5, "3": 0.7, 
                "4-1": 0.4, "4-2": 0.8, "4-3": 1, "5": 1}




df = pd.read_excel("Credit risk exposures and exposures to sovereigns a.xlsx", 
                   header=1, index_col=0)
df = df.iloc[:, 0:8]
df = df.drop("4. Total retail", axis = 1)
df.columns = ["1_Sov", "2_Fin", "3_Corp", "4-1_Ret_Res", "4-2_Ret_Rev", "4-3_Ret_SME", "5_CRE"]

bank_names_df = pd.read_csv("BankCodes.csv", index_col=0).iloc[:, 0].to_frame("Name")
bank_cap_df = pd.read_excel("Tier-1 CRs 2010.xlsx", 
                       header=2, index_col=1).iloc[:, 1].to_frame("Capital")
df = pd.concat([df, bank_names_df, bank_cap_df], axis=1)

sd_df = pd.read_excel("EU_SOV_Debt_Bank_Net_2011.xlsx", header=0, index_col=0).T
sd_df.loc[:, u"XX"] = np.maximum(df.iloc[:, 0] - sd_df.apply(np.sum, axis=1), 0)
sd_df.columns = sd_df.columns.rename("SD_Name")
sd_names = [sd.encode('ascii', 'ignore') for sd in sd_df.columns]


spread = [20, 55, 60, 130, 30, 5, 15, 40, 115, 10, 40, 25, 1400, 80, 315, 35,
          100, 20, 55, 40, 55, 50, 100, 20, 10, 50, 400, 70, 10, 30, 25, 20, 50]
spread_df = pd.DataFrame(data=spread, index=sd_names, columns=["spread"])
spread_df.loc[:, "Q"] = spread_df.loc[:, "spread"].apply(Q)

geo_df = sd_df.div(sd_df.apply(np.sum, axis=1), axis=0)

In [3]:
class World(object):
    'Common base class for all realizations of stress test'
    # To be passed: Dataframe containing asset classes, name, capital
    # To be passed: Dataframe containing how much banks are invested where
    def __init__(self, data, structure, spread, risk_dict):
        # Dataframe. Index: Banks, Columns: Asset classes, name, capital
        self._data = data
        # Geo-structure of banks' investments, usually inferred from sovereign debt
        self._structure = structure
        # Spreading parameters
        self._spread = spread
        # Risk dictionary
        self._risk_dict = risk_dict
        
        # Populate children
        self._banks  = self.initBanks(self._data)
        self._assets = self.initAssets(self.getByAsset())
        
        
    def __str__(self):
        return 'Implementation'        
    
    
    def initBanks(self, data):
        banks = {}
        for bank in data.index.values:
            banks[bank] = Bank(bank, 
                               self.getName(bank),
                               self.getCapital(bank))
        return banks
    
    
    def initAssets(self, data):
        assets = {}
        for asset in data.columns.values:
            assets[asset] = Asset(ExtractAssetClass(asset),
                                  asset,
                                  np.sum(data.loc[:, ]))
            
        for asset in assets:
            obj = assets[asset]
            obj_ac = obj.getAssetClass()
            obj.riskweight = self._risk_dict[obj_ac]
            if obj_ac == "1":
                obj.spread = self._spread.loc[obj.getCountry(), "Q"]
            else:
                obj.spread = 0.2
                
        return assets
    
    
    def getBanks(self):
        return self._banks
    
    
    def getAssets(self):
        return self._assets
    
    
    # All investments etc.
    def getByAssetClass(self, bank=None, asset_class=None):
        # List of columns in the data frame which are not asset-related
        droplist = ["Capital", "Name"]
        df = self._data
        
        for drop in droplist:
            try:
                df = df.drop(drop, axis=1)
            except:
                pass

        return dfSelecter(df, rows=bank, columns=asset_class)
        
        
    # Investments 
    def getByAsset(self, asset_class=None, bank=None, asset=None):
        # Data frame with banks' investments
        df = self.getByAssetClass(asset_class=asset_class)
        # Geo-structure of banks' investments
        weights_df = self._structure
        
        # Make sure we didn't pass a series
        if type(df)==pd.Series:
            df = df.to_frame()
        
        # Multiply each asset class to split up by country
        asset_classes = df.columns
        df_list = [weights_df.multiply(df.loc[:, ac], axis=0) 
                   for ac in asset_classes]
        
        # Concatenate data frames for each asset class and rename columns
        full_df = pd.concat(df_list, axis=1)
        full_df.columns = [col+"_"+ind for col in asset_classes 
                           for ind in weights_df.columns]
        return full_df
        
    
    # Read Capital information
    def getCapital(self, bank=None):    
        return dfSelecter(self._data, rows=bank, columns="Capital")

    
    # Names of Banks
    def getName(self, bank=None):    
        return dfSelecter(self._data, rows=bank, columns="Name")
    
    
    # Geo-structures of banks
    def getGeoStructure(self, bank=None):
        return dfSelecter(self._structure, rows=bank)


    
class Bank(World):
    'Class for banks in the model'
    # Initialize with name, country, holdings and capital ratio (since this is what we know)
    # When the stress test scenario is initialized, the capital is updated using the ratio 
    # and the RWA.
    
    def __init__(self, bankid, name, capital):
        self._bankid = bankid
        self._name = name
        self._capital = capital
        
    def __str__(self):
        return '{}, capital {}'.format(self._name, self._capital)

    
    # Name of the bank
    def getBankID(self):
        return self._bankid
    
    
    # Name of the bank
    def getName(self):
        return self._name

    def setName(self, n):
        self._name = n
        
    name = property(getName, setName)
    
    
    # WARNING: Due to the structure of the available data, 
    # initialization begins with ratio!
    def getCapital(self):
        return self._capital
    
    def setCapital(self, c):
        self._capital = c
        
    capital = property(getCapital, setCapital)
    
    
    def getHoldingsByAssetClass(self, asset_class=None):
        return parent.getByAssetClass(bank=bankid, asset_class=asset_class)
    
    
    def getHoldingsByAsset(self, asset_class=None, asset=None):
        return parent.getByAsset(bank=bankid, asset_class=asset_class, asset=asset)
    
    
    def getGeoStructure(self):
        return parent.getGeoStructure(bank=bankid)

    
    # EBA Bank ID
    def getBankID(self):
        return self._bankid
    
    
    # Country in which the bank is headquartered
    def getCountry(self):
        return self._bankid[0:2]
    
    
    
class Asset(World):
    'Class for assets in the model'
    # Initialize with asset class, name of asset, value, who holds it, but w/o risk weights
    def __init__(self, asset_class, name, value, spread=None, riskweight=None):
        self._asset_class = asset_class
        self._name = name
        self._country = name[-2:]
        self._value = value
        self._marketvalue = value
        self._spread = spread
        self._riskweight = riskweight
        self._staticriskweight = riskweight
        
    def __str__(self):
        return '{}, class {}, value {}, weight {}'.\
            format(self._name, self._asset_class, self._value, self._riskweight)
            
            
    # Name of asset
    def getName(self):
        return self._name
    
    
    # Country of asset, last two characters of name
    def getCountry(self):
        return self._country
    
    
    # Asset class, including sovereign debt, finance, commercial, retail and real estate.
    def getAssetClass(self):
        return self._asset_class
    
    
    # Total value of asset combining holdings of all banks
    def getValue(self):
        return self._value
    
        
    def getMarketValue():
        pass
    
    def setMarketValue(self, s):
        pass
        
    marketvalue = property(getMarketValue, setMarketValue)
    
        
    def getSpread(self):
        return self._spread
    
    def setSpread(self, s):
        self._spread = s
        
    spread = property(getSpread, setSpread)
    
    
    # Output risk weight or risk factor of asset at current moment
    def getRiskWeight(self):
        return self._riskweight
    
    # Function to update risk factor in crisis propagation
    def setRiskWeight(self, r):
        self._riskweight = r
        
    riskweight = property(getRiskWeight, setRiskWeight)

    
    # staticriskweight is the risk factor which was used to initialize asset
    def getStaticRiskWeight(self):
        return self._staticriskweight
        
    def setStaticRiskWeight(self, r):
        self._staticriskweight = r
        
    staticriskweight = property(getStaticRiskWeight, setStaticRiskWeight)    
    
    
    # Factor by which the risk factor has increased compared to initial state
    def getCrisisLevel(self):
        return self._riskweight/self._staticriskweight
    



In [4]:
world = World(df, geo_df, spread_df, getRiskWeights("min"))

assets = world.getAssets()
for asset in assets:
    print asset, assets.get(asset).getRiskWeight()

1_Sov_AT 0.002
2_Fin_GB 0.5
4-1_Ret_Res_SE 0.4
5_CRE_XX 1
4-1_Ret_Res_SK 0.4
4-1_Ret_Res_SI 0.4
3_Corp_MT 0.7
2_Fin_GR 0.5
4-2_Ret_Rev_SI 0.8
4-2_Ret_Rev_SK 0.8
4-2_Ret_Rev_SE 0.8
2_Fin_US 0.5
4-2_Ret_Rev_FR 0.8
4-2_Ret_Rev_HU 0.8
3_Corp_EE 0.7
1_Sov_IT 0.002
1_Sov_PL 0.002
4-3_Ret_SME_RO 1
4-1_Ret_Res_CZ 0.4
4-1_Ret_Res_CY 0.4
1_Sov_IS 0.002
3_Corp_ES 0.7
2_Fin_NL 0.5
2_Fin_NO 0.5
1_Sov_PT 0.002
4-2_Ret_Rev_AT 0.8
1_Sov_IE 0.002
4-3_Ret_SME_CZ 1
5_CRE_DE 1
4-3_Ret_SME_CY 1
4-1_Ret_Res_IE 0.4
1_Sov_FR 0.002
2_Fin_DK 0.5
4-1_Ret_Res_RO 0.4
4-3_Ret_SME_HU 1
3_Corp_NO 0.7
2_Fin_DE 0.5
5_CRE_LT 1
5_CRE_SI 1
4-1_Ret_Res_IT 0.4
4-1_Ret_Res_IS 0.4
1_Sov_FI 0.002
4-1_Ret_Res_JP 0.4
3_Corp_FI 0.7
4-2_Ret_Rev_CY 0.8
1_Sov_NL 0.002
1_Sov_NO 0.002
4-2_Ret_Rev_CZ 0.8
4-3_Ret_SME_SI 1
4-3_Ret_SME_SK 1
3_Corp_FR 0.7
4-1_Ret_Res_BE 0.4
2_Fin_FR 0.5
4-1_Ret_Res_BG 0.4
4-3_Ret_SME_LT 1
4-3_Ret_SME_MT 1
3_Corp_CY 0.7
4-2_Ret_Rev_IE 0.8
3_Corp_CZ 0.7
4-1_Ret_Res_LV 0.4
4-1_Ret_Res_LT 0.4
4-1_Ret_Res_LU 0.

In [5]:
print assets.get("1_Sov_GR").spread

0.999938964844
